# Gradient Descent

### Notes on Udacity

Gradient Descent with Squared Errors
We want to find the weights for our neural networks. Let's start by thinking about the goal. The network needs to make predictions as close as possible to the real values. To measure this, we use a metric of how wrong the predictions are, the error. A common metric is the sum of the squared errors (SSE):

$$E = \frac{1}{2}\sum_{\mu} \sum_j \left[ y^{\mu}_j - \hat{y} ^{\mu}_j \right]^2$$

where $\hat y $ is the prediction and $y$is the true value, and you take the sum over all output units jj and another sum over all data points $\mu$. This might seem like a really complicated equation at first, but it's fairly simple once you understand the symbols and can say what's going on in words.

In [2]:
import numpy as np
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [4]:
# input data

x =np.array([0.1, 0.3])

#target
y = 0.2

#input to output weights
weights = np.array([-0.8, 0.5])

# the learning rate, eta in the weight step equation
learnrate = 0.5

# the linear combination performed by the node (h in f(h) and f'(h))
h = np.dot(x, weights)

# the Neural Network output y-hat
nn_output = sigmoid(h)

# output error (y - y_hat)
error = y - nn_output

# output gradient (f'(h))
output_grad = sigmoid_prime(h)

# error_term (lowercase delta)
error_term = error * output_grad

# Gradient descent step 
del_w = [ learnrate * error_term * x[0],
          learnrate * error_term * x[1]]
# or learnrate * error_term * x

# Example implementing gradient descent

Okay, now we know how to update our weights (example above) and formula:

$$\nabla{w_{ij}} = \eta * \delta_{j} * x_{i}$$

You've seen how to implement that for a single update, but how do we translate that code to calculate many weight updates so our network will learn?

We will train a network on graduate school admissions data found at http://www.ats.ucla.edu/stat/data/binary.csv. 

This dataset has three input features, GRE Score, GPA, and the rank of the undergraduate school (numbered 1 through 4). Institutions with rank 1 have the highest prestige, those with rank 4 have the lowest. 

The goal here is to predict if a student will be admitted to a graduate program based on these features. For this, we will use a network with one output layer and one unit. We will use a sigmoid function for the output unit activation. 

## Data Cleanup 

You might think there will be three input units, but we actually need to transform the data first. The rank feature is categorical, the numbers do not encode any sort of relative value. Rank 2 is not twice as much as rank 1, rank 3 is not 1.5 more than rank 2. 

Instead, we will use dummy variables to encode the rank, splitting the data into four new columns encoded with ones or zeros. Rows with rank 1 will have one in the rank 1 dummy column, and zeros in all other columns. This will go for each rank number etc.

We will also need to standardize the GRE and GPA data, which means to scale the values such that they have zero mean and a standard deviation of 1. This is necessary because the sigmoid funtion squashes really small and really large inputs. The gradient of really small and large inputs is zero, which means that the gradient descent step will go to zero too. Since the GRE and GPA values are fairly large, we will have to be really careful about how we initalize the weights or the gradient descent steps will die off and the network won't train. Instead, if we standardize the data, we can initialize the weights easily and everyone is happy. 

**Mean Square Error**

We're going to make a small change to how we calculate the error here. Instead of the SSE, we're going to use the **mean** of the square errors (MSE). Now that we're using a lot of data, summing up all the weight steps can lead to really large updates that make the gradient descent diverge. To compensate for this, you'd need to use a quite small learning rate. Instead, we can just divide by the number of records in our data, $m$ to take the average. This way, no matter how much data we use, our learning rates will typically be in the range of 0.01 to 0.001. Then, we can use the MSE (shown below) to calculate the gradient and the result is the same as before, just averaged instead of summed. 

$$ E = \frac{1}{2m} \sum_{\mu} \left(y^\mu - \hat{y}_{\mu} \right)^2 $$

Here's the general algorithm for updating the weights with gradient descent.

1) Set the weight step to zero: $\nabla w_i = 0$

2) For each record in the training data:
- Make a forward pass through the network, calculating the output
   
    $$\hat{y} = f\left(\sum_i w_i x_i \right)$$
- Calculate the error term for the output unit

$$ \delta = \left(y - \hat{y} \right) * f^\prime \left(\sum_i w_i x_i \right) $$

- Update the weight step $\nabla w_i = \nabla w_i + \delta x_i$

3) Update the weights $w_i = w_i + \eta \nabla w_i / m$ where $\eta$ is the learning rate and m is the number of records. Here we're averaging the weight steps to help reduce any large variations in the training data.

4) Repeat for e epochs.


### Implementing with NumPy

First, you'll need to initalize the weights. We want these to be small such that the input to the sigmoid is in the linear region near 0 and not squashed at the high and low ends. It's also important to initialize them randomly so that they all have different starting values and diverge, breaking symmetry. So we'll initialize the weights from a normal distribution centered at 0. A good value for the scale is $ 1 / \sqrt{n}$ where n is the number of input units. This keeps the input to the sigmoid low for increasing numbers of input units.


In [5]:
import numpy as np
n_features = 8
weights = np.random.normal(scale = 1 / n_features **.5, size=n_features)

NumPy provides a function np.dot() that calculates the dot product of two arrays, which conveniently calculates $h$ for us. The dot product multiplies two arrays element-wise, the first element in array 1 is multiplied by the first element in array 2, and so on. Then, each product is summed. 

In [8]:
#ouput_in = np.dot(weights, inputs)

And finally, we can update $\nabla w_i$ and $w_i$ by incrementing them with weights += ....

### Efficiency tip!

You can save some calculations since we're using a sigmoid here. For the sigmoid function, $f^\prime(h) = f(h)(1-f(h))$ That means that once you calculate $f(h)$, the activation of the output unit, you can use it to calculate the gradient for the error gradient.

# Example: Admissions Data

Here we will do the example in Gradient Descent lesson in Deep Learning Nanodegree.


### Data Prep

In [17]:
import numpy as np
import pandas

admissions = pandas.read_csv('../data/binary.csv')

In [19]:
# Make dummy variables for rank
data = pandas.concat([admissions, pandas.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data.drop('rank', axis=1, inplace=True)

In [20]:
# Standardize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field] - mean) / std

In [21]:
# split off random_10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

#Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


### Gradient Steps

Programming exercise
Below, you'll implement gradient descent and train the network on the admissions data. Your goal here is to train the network until you reach a minimum in the mean square error (MSE) on the training set. You need to implement:

- The network output: output.
- The output error: error.
- The error term: error_term.
- Update the weight step: del_w +=.
- Update the weights: weights +=.

After you've written these parts, run the training by pressing "Test Run". The MSE will print out, as well as the accuracy on a test set, the fraction of correctly predicted admissions.

Feel free to play with the hyperparameters and see how it changes the MSE.

In [23]:
def sigmoid(x):
    return 1 /(1+np.exp(-x))

In [22]:
# Set seed, records, features, and last loss
np.random.seed(18)
n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

In [25]:
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # loop through all records, x is the input, y is the target
        
        # we have not included the h variable, as we did in other lessons
        # we can add h if we want
        # h = np.dot(weights, x)
        
        # the output is the sigmoid of h
        # Rather than storing h as a separate variable, 
        # we multiply the inputs and the weights here
        output = sigmoid(np.dot(weights, x))
        
        # calculate the error
        error = y - output
        
        # calculate the error term
        error_term = error * output * (1 - output)
        
        # calculate the change in weights for this sample
            # and add it to the total weight change
        del_w += error_term * x
        
    weights += learnrate * del_w / n_records
    
    # printing out the mean square error on the training data set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out-targets) ** 2)
        if last_loss and last_loss < loss:
            print('Train loss:', loss, " WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# calculate accuracy on test data
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction Accuracy: {:.3f}".format(accuracy))

Train loss:  0.2659720509102629
Train loss:  0.2127815193115812
Train loss:  0.20225122157508604
Train loss:  0.1992456789974773
Train loss:  0.19810007767962187
Train loss:  0.19758332928381614
Train loss:  0.19732274589024312
Train loss:  0.19718067213447626
Train loss:  0.19709871014961833
Train loss:  0.19704941361886977
Prediction Accuracy: 0.725


## Multi-layer Perceptrons

Before we were dealing with only one output node which made the code straightforward. However now that we have multiple input units and multiple hidden units, the weights between them will require two indices: $w_{ij}$ where i denotes input units and $j$ are the hidden units. 

For example, the following image shows our network, with its input units labeled $x_1, x_2$ and $x_3$, and its hidden notes labeled $h_1$ and $h_2$

![Weights Network](..\data\weights_network1.png)

Before we were able to write the weights as an array, indexed as $wi$. 

But now, the weights need to be stored in a **matrix**, indeed as $w_{ij}$. Each **row** in the matrix will correspond to the weights leading out of a single input unit, and for each column will correspond to the weights leading in to a **single hidden unit**. For our three input units and two hidden units, the weights matrix looks like this:

$ = \left[
\begin{array}{cc}
    w_{11} w_{12}\\
    w_{21} w_{22}\\
    w_{31} w_{32}
\end{array}
\right]$

So $h_1$ to $x_1$ would be $w_{11}$, and $x_2$ would be $w_{21}$, while $h_2$ to $x_1$ would be $w_{12}$, while to $x_2$ would be $w_{22}$

![title](..\data\weights_network.png)

Be sure to compare the matrix above with the diagram shown before it so you can see where the different weights in the network end up in the matrix.

To initialize these weights in NumPy, we have to provide the shape of the matrix. If features is a 2D array containing the input data

In [ ]:
# number of records and input units
n_records, n_inputs = features.shape

# number of hidden units
n_hidden = 2
weights_input_to_hidden = np.random.normal(0, n_inputs**-0.5, size=(n_inputs, n_hidden))

This creates a 2D array (i.e. a matrix) named weights_input_to_hidden with dimension n_inputs by n_hidden. Remember how the input to a hidden unit is the sum of all the inputs multiplies by the hidden units weights. So for each hidden layer unit, $h_j$, we need to calculate the following:

$$ h_j = \sum_i w_{ij}x_i$$

To do that we now use matrix multiplication. In this case, we are multiplying the inputs (a row vector here) by the weights. To do this, you take the dot (inner) product of the inputs with each column in the weights matrix. For example, to calculate the input to the first hidden unit, $j=1$, you'd take the dot product of the inputs with the first column of the weights matrix, like so:

![first hidden](..\data\weights_firsthidden.png)

**Calculating the input to the first hidden unit with the first column of the weights matrix.**

$$h_1 = x_1 w_{11} + x_2 w_{21} + x_3 w_{31}$$

And for the second hidden layer input, you calculate the dot product of the inputs with the second column. And so on and so forth. 

In NumPy, you can do this for all the inputs and all the outputs at once using np.dot




In [ ]:
hiden_inputs = np.dot(inputs, weights_input_to_hidden)

You could also define your weights matrix such that it has dimensions n_hidden by n_inputs then multiply like so where the inputs form a column vector:

![hj](..\data\hj.png)

**Note: ** The weight indices have changed in the above image and no longer match up with the labels used in earlier diagrams. That's beacuse, in matrix notation, **the row index always precedes the column index**, so it would be misleading to label them the way we did in the neural net diagram. Just keep in mind that this is the same weight matrix as before, but rotated so the first column is now the first row, and the second column is now the second row.

The important thing with matrix multiplication is that _the dimension match_.



The dot product can't be computed for a 3x2 matrix and 3-element array. That's because the 2 columns in the matrix don't match the number of elements in the array. 

The rule is that if you're multiplying an array from the left, the array must have the same number of elements as there are rows in the matrix. And if you're multiplying the matrix from the left, the number of columns in the matrix must equal the number of elements in the array on the right.

## Making a column vector

You see above that sometimes you'll want a column vector, even though by default NumPy arrays work like row vectors. It's possible to get the transpose of an array like so arr.T, byt for a 1D array, the transpose will return a row vector. Instead use arr[:,None] to create a column vector:



# Example: Multi-layer Perceptrons

Programming quiz
Below, you'll implement a forward pass through a 4x3x2 network, with sigmoid activation functions for both layers.

Things to do:

1. Calculate the input to the hidden layer.
2. Calculate the hidden layer output.
3. Calculate the input to the output layer.
4. Calculate the output of the network.

In [27]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

# Network size
N_input = 4
N_hidden = 3
N_output = 2

np.random.seed(42)
# Make some fake data
X = np.random.randn(4)

weights_input_to_hidden = np.random.normal(0, scale=0.1, size=(N_input, N_hidden))
weights_hidden_to_output = np.random.normal(0, scale=0.1, size=(N_hidden, N_output))


# TODO: Make a forward pass through the network

hidden_layer_in = np.dot(X, weights_input_to_hidden)
hidden_layer_out = sigmoid(hidden_layer_in)

print('Hidden-layer Output:')
print(hidden_layer_out)

output_layer_in = np.dot(hidden_layer_out, weights_hidden_to_output)
output_layer_out = sigmoid(output_layer_in)

print('Output-layer Output:')
print(output_layer_out)

Hidden-layer Output:
[0.41492192 0.42604313 0.5002434 ]
Output-layer Output:
[0.49815196 0.48539772]


# Backpropagation

Now we've come to the problem of how to make a multilayer neural network _learn_. Before, we saw how to update weights with gradient descent. THe backpropagation algorithm is just an extension of that, using the chain rule to find the error with respect to the weights connecting the input layer to the hidden layer (for a two layer network). 

To update the weights to hidden layers using gradient descent, you need to know how much error each of the hidden units contributed to the final output. Since the output of a layer is determined by the weights between layers, the error resulting from units is scaled by the weights going forward through the network. Since we know the error at the output, we can use the weights to work backwards to hidden layers. 

For example, in the output layer, you have errors $\delta_k^o$ attributed to each output unit $k$. Then , the error attributed to hidden unit $j$ is the output errors, scaled by the weights between the output and hidden layers (and the gradient):

$$\delta_j^h = \sum W_{jk} \delta_k^o f^\prime (h_j)$$

Then, the gradient descent step is the same as before, just with new errors:

$$\nabla w_{ij} = \eta \delta_j^h x_i $$

where $w_{ij}$ are the weights between the inputs and hidden layer and $x_i$ are input unit values. This form holds for however many layers there are. The weight steps are equal to the step size times the output error of the layer times the values of the inputs to that layer. 

$$\nabla_{pq} = \eta\delta_{output}V_{in}$$

Here, you get the output error, $\delta_{output}$, by propagating the error backwards from the higher layers. And the input values, $V_{in}$ are the inputs to the layer, the hidden layer activations to the output unit for example. 

## Working through an example

Let's walk through the steps of calcuating the weight updates for a simple two layer network. Suppose there are two input values, one hidden unit, and one output unit, with sigmoid activations on the hidden and output units. The following imade depits the network (**Note:** the input values are shown as nodes as the bottom of the imade, while the network's output value is shown as $\hat{y}$ at the top. The inputs themselves do not count as a layer, which is why this is considered a two layer network. 

![network](..\data\network.png)

Assume we're trying to fit some binary data and the target is y = 1, We'll start with the forward pass, first calculating the input to the hidden unit

$ h = \sum_i w_i x_i = 0.1 \times 0.4 - 0.2 \times 0.3 = -0.02$

and the **output of the hidden units**

$ \alpha = f(h) = sigmoid(-0.02) = 0.495$

Using this as the input to the output unit, the **output of the network** is 

$\hat{y} = f(W \cdot a) = sigmoid(0.1 \times 0.495) = 0.512$

With the network output, we can start the backwards pass to calculate the weight updates for both layers. Using the fact that the sigmoid function 

$f^\prime(W \cdot a) = f(W \cdot a)(1 - f(W \cdot a))$, the error term for the **output unit** is

$\delta^o = (y - \hat{y})f^\prime(W\cdot a) = (1 - 0.512) \times 0.512 \times (1 - 0.512) = 0.122$

Now we need to calculate the error term for the hidden unit with backpropogation. Here we'll scale the error term from the output unit by the weight **W** connecting it to the hidden unit. For the hidden unit error term, $\delta_j^h = \sum_k W_{jk} \delta_k^o f^\prime (h_j)$, but since we have one hidden unit and one output unit, this is much simpler

$\delta^h = W \delta^o f^\prime(h) = 0.1 \times 0.122 \times 0.495 \times (1 - 0.495) = 0.003$

Now that twe have the errors, we can calculate the gradient descent steps. The hidden to output weight step is the learning rate, times the output unit error, times the hidden unit activation value. 

$\nabla W = \eta \delta^o \alpha = 0.5 \times 0.122 \times 0.495 = 0.0302$

Then, for the input to hidden weights $w_i$, it's the learning rate times the hidden unit error, times the input values. 

$\nabla w_i = \eta \delta^h x_i = (0.5 \times 0.003 \times 0.1, 0.5 \times 0.003 \times 0.3) = (0.00015, 0.00045)$

From this example, you can see one of the effects of using the sigmoid function for the activations. The maximum derivation of the sigmoid function is 0.25, so the errors in the output layer get reduced by at least 75%, and errors in the hidden layer are scaled down by at least 93.75%! You can see that if you have a lot of layers, using a sigmoid activation function will quickly reduce the weight steps to tiny values in layers near the input. This is known as the **vanishing gradient** problem. Later in the course you'll learn about other activation functions that perform better in this regard and are more commonly used in modern network architectures. 



## Implementing in NumPy

For the most we have everything we need, 

### Backpropagation exercise
Below, you'll implement the code to calculate one backpropagation update step for two sets of weights. I wrote the forward pass - your goal is to code the backward pass.

#### Things to do

1. Calculate the network's output error.
2. Calculate the output layer's error term.
3. Use backpropagation to calculate the hidden layer's error term.
4. Calculate the change in weights (the delta weights) that result from propagating the errors back through the network.

In [28]:
import numpy as np


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5

weights_input_hidden = np.array([[0.5, -0.6],
                                 [0.1, -0.2],
                                 [0.1, 0.7]])

weights_hidden_output = np.array([0.1, -0.3])

## Forward pass
hidden_layer_input = np.dot(x, weights_input_hidden)
hidden_layer_output = sigmoid(hidden_layer_input)

output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)
output = sigmoid(output_layer_in)

## Backwards pass
## TODO: Calculate output error
error = target - output

# TODO: Calculate error term for output layer
output_error_term = error * output * (1 - output)

# TODO: Calculate error term for hidden layer
hidden_error_term = np.dot(output_error_term, weights_hidden_output) * \
hidden_layer_output * (1 - hidden_layer_output) 

# TODO: Calculate change in weights for hidden layer to output layer
delta_w_h_o = learnrate * output_error_term * hidden_layer_output

# TODO: Calculate change in weights for input layer to hidden layer
delta_w_i_h = learnrate * hidden_error_term * x[:, None]

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)


Change in weights for hidden layer to output layer:
[0.00804047 0.00555918]
Change in weights for input layer to hidden layer:
[[ 1.77005547e-04 -5.11178506e-04]
 [ 3.54011093e-05 -1.02235701e-04]
 [-7.08022187e-05  2.04471402e-04]]


## Implementing backpropagation

Now we've seen that the error term for the output layer is 

$\delta_k = (y_k - \hat{y}_k) f^\prime(\alpha_k)$

and the error term for the hidden layer is 

$$ \delta_j = \sum \left[w_{jk}\delta_k\right] f^\prime(h_j)$$

For now we'll only consider a simple network with one hidden layer and one output unit. Here's the general algorithm for updating the weights with backpropagation: 

**First, Set the weight steps for each layer to zero**

- The input to hidden weights $\nabla w_{ij} = 0$
- The hidden to output weights $\nabla W_j = 0$

**Second, For each record in the training data:**

- Make a forward pass through the network, calculating the output $\hat{y}$
- Calculate the error gradient in the output unit, $\delta^o = (y - \hat{y})\ f^\prime (z)$ where $z = \sum_j W_j \alpha_j$, the input to the output unit. 
- Propagate the errors to the hidden layer $\delta_j^h = \delta^o W_i \ f^\prime(h_j)$
- Update the weight steps: 
    - $\nabla W_j = \nabla W_j + \delta^o \alpha_j$
    - $\nabla w_{ij} = \nabla_{ij} + \delta_j^h \alpha_i$
- Update the weights, where $\eta$ is the learning rate and $m$ is the number of records:
    - $W_j = W_j + \eta\nabla W_j / m$
    - $w_{ij} = w_{ij} + \eta\nabla w_{ij} / m$
    
    
- Repeat for $e$ epochs.

    

## Backpropagation exercise
Now you're going to implement the backprop algorithm for a network trained on the graduate school admission data. You should have everything you need from the previous exercises to complete this one.

**Your goals here:**

- Implement the forward pass.
- Implement the backpropagation algorithm.
- Update the weights.

In [31]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('../data/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [39]:
np.random.seed(18)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2 # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# initalize weights

weights_input_hidden = np.random.normal(scale=1 / n_features ** .5, 
                                       size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features **.5,
                                        size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        
        ## Forward Pass ##
        # Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))
        
        ## Backward Pass ##
        # Calculate the network's prediction error
        error = y - output
        
        # Calculate error term for the output unit
        output_error_term = error * output * (1 - output)
        
        ## propagate errors to hidden layer
        
        # Calculate the hidden layer's contribution to the error 
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        
        # Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output) 
        
        # Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:, None]
        
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records
    
    if e % (epochs / 10) == 0:
        hidden_ouput = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                            weights_hidden_output))
        loss = np.mean((out-targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

        

Train loss:  0.2529161892134775
Train loss:  0.2521576505212607
Train loss:  0.2514181077544339
Train loss:  0.2506969893839888
Train loss:  0.2499937410828779
Train loss:  0.24930782538355514
Train loss:  0.24863872132151002
Train loss:  0.24798592406752137
Train loss:  0.2473489445511789
Train loss:  0.24672730907792267
Prediction accuracy: 0.650


# Further Reading

1. https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b

2. https://www.youtube.com/watch?v=59Hbtz7XgjM
